### Meeting a Sayed Athar's request, I'm using the Kernel altered by Khoi Nguyen to explain how the whole code works.
### If any part is not clear, please comment.  
### Please upvote if it was helpful.

In [1]:
import pandas as pd
import pyarrow.parquet as pq # Used to read the data
import os 
import numpy as np
from keras.layers import * # Keras is the most friendly Neural Network library, this Kernel use a lot of layers classes
from keras.models import Model
from tqdm import tqdm # Processing time measurement
from sklearn.model_selection import train_test_split 
from keras import backend as K # The backend give us access to tensorflow operations and allow us to create the Attention class
from keras import optimizers # Allow us to access the Adam class to modify some parameters
from sklearn.model_selection import GridSearchCV, StratifiedKFold # Used to use Kfold to train our model
from keras.callbacks import * # This object helps the model to train in a smarter way, avoiding overfitting

from scipy.signal import chirp, find_peaks, peak_widths
import pywt

from multiprocessing import Pool
from sklearn.preprocessing import MinMaxScaler, StandardScaler

import warnings

warnings.filterwarnings('ignore')

/home/cwademo123/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# select how many folds will be created
N_SPLITS = 5
# it is just a constant with the measurements data size
sample_size = 800000

In [3]:
def matthews_correlation(y_true, y_pred):
    '''Calculates the Matthews correlation coefficient measure for quality
    of binary classification problems.
    '''
    y_pred_pos = K.round(K.clip(y_pred, 0, 1))
    y_pred_neg = 1 - y_pred_pos

    y_pos = K.round(K.clip(y_true, 0, 1))
    y_neg = 1 - y_pos

    tp = K.sum(y_pos * y_pred_pos)
    tn = K.sum(y_neg * y_pred_neg)

    fp = K.sum(y_neg * y_pred_pos)
    fn = K.sum(y_pos * y_pred_neg)

    numerator = (tp * tn - fp * fn)
    denominator = K.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))

    return numerator / (denominator + K.epsilon())

In [4]:
# https://www.kaggle.com/suicaokhoailang/lstm-attention-baseline-0-652-lb

class Attention(Layer):
    def __init__(self, step_dim,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, x, mask=None):
        features_dim = self.features_dim
        step_dim = self.step_dim

        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)),
                        K.reshape(self.W, (features_dim, 1))), (-1, step_dim))

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        if mask is not None:
            a *= K.cast(mask, K.floatx())

        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0],  self.features_dim

In [5]:
# just load train data
df_train = pd.read_csv('../input/metadata_train.csv')
# set index, it makes the data access much faster
df_train = df_train.set_index(['id_measurement', 'phase'])
df_train.head()

signal_id  target
id_measurement phase                   
0              0              0       0
               1              1       0
               2              2       0
1              0              3       1
               1              4       1

In [6]:
# in other notebook I have extracted the min and max values from the train data, the measurements
max_num = 127
min_num = -128

In [7]:
# This function standardize the data from (-128 to 127) to (-1 to 1)
# Theoretically it helps in the NN Model training, but I didn't tested without it
def min_max_transf(ts, min_data, max_data, range_needed=(-1,1)):
    if min_data < 0:
        ts_std = (ts + abs(min_data)) / (max_data + abs(min_data))
    else:
        ts_std = (ts - min_data) / (max_data - min_data)
    if range_needed[0] < 0:    
        return ts_std * (range_needed[1] + abs(range_needed[0])) + range_needed[0]
    else:
        return ts_std * (range_needed[1] - range_needed[0]) + range_needed[0]

In [8]:
def maddest(d, axis=None):
    """
    Mean Absolute Deviation
    """
    return np.mean(np.absolute(d - np.mean(d, axis)), axis)

In [9]:
def denoise_signal( x, wavelet='db4', level=1):
    """
    1. Adapted from waveletSmooth function found here:
    http://connor-johnson.com/2016/01/24/using-pywavelets-to-remove-high-frequency-noise/
    2. Threshold equation and using hard mode in threshold as mentioned
    in section '3.2 denoising based on optimized singular values' from paper by Tomas Vantuch:
    http://dspace.vsb.cz/bitstream/handle/10084/133114/VAN431_FEI_P1807_1801V001_2018.pdf
    """
    
    # Decompose to get the wavelet coefficients
    coeff = pywt.wavedec( x, wavelet, mode="per", level=level)
    
    # Calculate sigma for threshold as defined in http://dspace.vsb.cz/bitstream/handle/10084/133114/VAN431_FEI_P1807_1801V001_2018.pdf
    # As noted by @harshit92 MAD referred to in the paper is Mean Absolute Deviation not Median Absolute Deviation
    sigma = (1/0.6745) * maddest( coeff[-level] )

    # Calculte the univeral threshold
    uthresh = sigma * np.sqrt( 2*np.log( len( x ) ) )
    coeff[1:] = ( pywt.threshold( i, value=uthresh, mode='hard' ) for i in coeff[1:] )
    
    # Reconstruct the signal using the thresholded coefficients
    return pywt.waverec( coeff[1:], wavelet, mode='per' )

In [10]:
def remove_corona(x_dn, maxDistance=10, maxHeightRatio=0.25, maxTicksRemoval=500):
    index = pd.Series(x_dn).loc[np.abs(x_dn)>0].index
    corona_idx = []
    for idx in index:
        for i in range(1,maxDistance+1):
            if idx+i < pd.Series(x_dn).shape[0]:
                if x_dn[idx+i]/(x_dn[idx]+1e-04)<-maxHeightRatio:
                    x_dn[idx:idx+maxTicksRemoval] = 0
                    corona_idx.append(idx)
    return x_dn, corona_idx

In [11]:
# This is one of the most important peace of code of this Kernel
# Any power line contain 3 phases of 800000 measurements, or 2.4 millions data 
# It would be praticaly impossible to build a NN with an input of that size
# The ideia here is to reduce it each phase to a matrix of <n_dim> bins by n features
# Each bean is a set of 5000 measurements (800000 / 160), so the features are extracted from this 5000 chunk data.
def transform_ts(ts, n_dim=160, min_max=(-1,1)):
    # convert data into -1 to 1
    ts_std = min_max_transf(ts, min_data=min_num, max_data=max_num)
    # bucket or chunk size, 5000 in this case (800000 / 160)
    bucket_size = int(sample_size / n_dim)
    # new_ts will be the container of the new data
    ts_wave = denoise_signal(ts_std, wavelet='haar', level=1)
    ts_rm, corona_idx = remove_corona(ts_wave)
    
    new_ts = []
    # this for iteract any chunk/bucket until reach the whole sample_size (800000)
    for i in range(0, sample_size, bucket_size):
        # cut each bucket to ts_range
        ts_range = ts_std[i:i + bucket_size]
        ts_rm_range = ts_rm[int(i/2):int(i/2) + int(bucket_size/2)]
        #ts_rm_range = pd.Series(ts_rm_range)
        
        # calculate each feature
        mean = ts_range.mean()
        std = ts_range.std() # standard deviation
        std_top = mean + std # I have to test it more, but is is like a band
        std_bot = mean - std
        # I think that the percentiles are very important, it is like a distribuiton analysis from eath chunk
        percentil_calc = np.percentile(ts_range, [0, 1, 25, 50, 75, 99, 100]) 
        max_range = percentil_calc[-1] - percentil_calc[0] # this is the amplitude of the chunk
        relative_percentile = percentil_calc - mean # maybe it could heap to understand the asymmetry
        
        #numpeaks = ts_rm_range[ts_rm_range!=0].count()
        numpeaks = np.sum(ts_rm_range!=0)/len(ts_rm_range)
        #numpospeaks = ts_rm_range[ts_rm_range>0].count()
        numpospeaks = np.sum(ts_rm_range>0)/len(ts_rm_range)
        #numnegpeaks = ts_rm_range[ts_rm_range<0].count()
        numnegpeaks = np.sum(ts_rm_range<0)/len(ts_rm_range)

        meanamp = np.mean(ts_rm_range)
        meanamppos = np.mean(ts_rm_range[ts_rm_range>0])
        meanampneg = np.mean(ts_rm_range[ts_rm_range<0])

        maxamp = np.max(ts_rm_range)
        minamp = np.min(ts_rm_range)
        
        peaks, _ = find_peaks(ts_rm_range)
        results_full = peak_widths(ts_rm_range, peaks, rel_height=1)
        
        if len(results_full[0])==0:
            maxwidth = 0
            minwidth = 0
        else:
            maxwidth = np.max(results_full[0])/len(ts_rm_range)
            minwidth = np.min(results_full[0])/len(ts_rm_range)
        
        
        
        feat_array = np.asarray([mean, std, std_top, std_bot, max_range])
        feat_array_2 = np.asarray([numpeaks, numpospeaks, numnegpeaks
                                   , meanamp, meanamppos, meanampneg,
                                 maxamp, minamp, maxwidth, minwidth
                                  ])
        
        new_ts.append(np.concatenate([feat_array, feat_array_2, percentil_calc, relative_percentile]))
        
    #scaler = MinMaxScaler(feature_range=(-1, 1))
    #scaler = StandardScaler()
    #new_ts = np.asarray(new_ts)
    #new_ts[np.isnan(new_ts)] = 0
    
    #new_ts = scaler.fit_transform(new_ts)
        
    return new_ts

In [16]:
# This is one of the most important peace of code of this Kernel
# Any power line contain 3 phases of 800000 measurements, or 2.4 millions data 
# It would be praticaly impossible to build a NN with an input of that size
# The ideia here is to reduce it each phase to a matrix of <n_dim> bins by n features
# Each bean is a set of 5000 measurements (800000 / 160), so the features are extracted from this 5000 chunk data.
def transform_ts_sum(ts, n_dim=160, min_max=(-1,1)):
    # convert data into -1 to 1
    ts_std = min_max_transf(ts, min_data=min_num, max_data=max_num)
    # bucket or chunk size, 5000 in this case (800000 / 160)
    bucket_size = int(sample_size / n_dim)
    # new_ts will be the container of the new data
    #ts_wave = denoise_signal(ts_std, wavelet='haar', level=1)
    #ts_rm, corona_idx = remove_corona(ts_wave)
    
    new_ts = []
    # this for iteract any chunk/bucket until reach the whole sample_size (800000)
    for i in range(0, sample_size, bucket_size):
        # cut each bucket to ts_range
        ts_range = ts_std[i:i + bucket_size]
        #ts_rm_range = ts_rm[int(i/2):int(i/2) + int(bucket_size/2)]
        #ts_rm_range = pd.Series(ts_rm_range)
        
        # calculate each feature
        mean = ts_range.mean()
        std = ts_range.std() # standard deviation
        std_top = mean + std # I have to test it more, but is is like a band
        std_bot = mean - std
        # I think that the percentiles are very important, it is like a distribuiton analysis from eath chunk
        percentil_calc = np.percentile(ts_range, [0, 1, 25, 50, 75, 99, 100]) 
        max_range = percentil_calc[-1] - percentil_calc[0] # this is the amplitude of the chunk
        relative_percentile = percentil_calc - mean # maybe it could heap to understand the asymmetry
        
        #numpeaks = ts_rm_range[ts_rm_range!=0].count()
        numpeaks = np.sum(ts_range!=0)/len(ts_range)
        #numpospeaks = ts_rm_range[ts_rm_range>0].count()
        numpospeaks = np.sum(ts_range>0)/len(ts_range)
        #numnegpeaks = ts_rm_range[ts_rm_range<0].count()
        numnegpeaks = np.sum(ts_range<0)/len(ts_range)

        #meanamp = np.mean(ts_rm_range)
        meanamppos = np.mean(ts_range[ts_range>0])
        meanampneg = np.mean(ts_range[ts_range<0])

        #maxamp = np.max(ts_rm_range)
        #minamp = np.min(ts_rm_range)
        
        peaks, _ = find_peaks(ts_range)
        results_full = peak_widths(ts_range, peaks, rel_height=1)
        
        if len(results_full[0])==0:
            maxwidth = 0
            minwidth = 0
        else:
            maxwidth = np.max(results_full[0])/len(ts_range)
            minwidth = np.min(results_full[0])/len(ts_range)
        
        
        
        feat_array = np.asarray([mean, std, std_top, std_bot, max_range])
        feat_array_2 = np.asarray([numpeaks, numpospeaks, numnegpeaks
                                   , meanamppos, meanampneg,
                                 maxwidth, minwidth
                                  ])
        
        new_ts.append(np.concatenate([feat_array, feat_array_2, percentil_calc, relative_percentile]))
        
    #scaler = MinMaxScaler(feature_range=(-1, 1))
    #scaler = StandardScaler()
    #new_ts = np.asarray(new_ts)
    #new_ts[np.isnan(new_ts)] = 0
    
    #new_ts = scaler.fit_transform(new_ts)
        
    return new_ts

In [22]:
# this function take a piece of data and convert using transform_ts(), but it does to each of the 3 phases
# if we would try to do in one time, could exceed the RAM Memmory
def prep_data(start, end):
    # load a piece of data from file
    praq_train = pq.read_pandas('../input/train.parquet', columns=[str(i) for i in range(start, end)]).to_pandas()
    X = []
    y = []
    # using tdqm to evaluate processing time
    # takes each index from df_train and iteract it from start to end
    # it is divided by 3 because for each id_measurement there are 3 id_signal, and the start/end parameters are id_signal
    for id_measurement in tqdm(df_train.index.levels[0].unique()[int(start/3):int(end/3)]):
        X_signal = []
        # for each phase of the signal
        for phase in [0,1,2]:
            # extract from df_train both signal_id and target to compose the new data sets
            signal_id, target = df_train.loc[id_measurement].loc[phase]
            # but just append the target one time, to not triplicate it
            if phase == 0:
                y.append(target)
                ts_sum = praq_train[str(signal_id)] + praq_train[str(signal_id+1)] + praq_train[str(signal_id+2)]
                X_signal.append(transform_ts_sum(ts_sum))
                
            # extract and transform data into sets of features
            #X_signal.append(transform_ts(praq_train[str(signal_id)]))
        # concatenate all the 3 phases in one matrix
        X_signal = np.concatenate(X_signal, axis=1)
        # add the data to X
        X.append(X_signal)
    X = np.asarray(X)
    y = np.asarray(y)
    return X, y

In [14]:
praq_train = pq.read_pandas('../input/train.parquet', columns=[str(i) for i in range(0, 3)]).to_pandas()

In [18]:
np.max(praq_train['0']+praq_train['1']+praq_train['2'])

45

In [19]:
np.min(praq_train['0']+praq_train['1']+praq_train['2'])

-46

In [14]:
def process_subtrain(arg_tuple):
    start, end, idx = arg_tuple
    X, y = prep_data(start, end)
    return idx, X, y

In [23]:
# this code is very simple, divide the total size of the df_train into two sets and process it
#X = []
#y = []
all_chunks = []

num_cores = 8 
#def load_all():
total_size = len(df_train)
chunk_size = total_size/num_cores

for i in range(8):
    start_idx = int(i * chunk_size)
    end_idx = int(start_idx + chunk_size)
    chunk = (start_idx, end_idx, i)
    all_chunks.append(chunk)

pool = Pool()
results = pool.map(process_subtrain, all_chunks)    
results = sorted(results, key=lambda tup: tup[0])

X = np.concatenate([item[1] for item in results], axis=0)
y = np.concatenate([item[2] for item in results], axis=0)

#load_all()

#X = np.asarray(X)
#y = np.asarray(y)

100%|██████████| 363/363 [04:37<00:00,  1.69it/s]


In [24]:
# The X shape here is very important. It is also important undertand a little how a LSTM works
# X.shape[0] is the number of id_measuremts contained in train data
# X.shape[1] is the number of chunks resultant of the transformation, each of this date enters in the LSTM serialized
# This way the LSTM can understand the position of a data relative with other and activate a signal that needs
# a serie of inputs in a specifc order.
# X.shape[3] is the number of features multiplied by the number of phases (3)
print(X.shape, y.shape)

(2904, 1, 160, 26) (2904,)


In [25]:
X = X.reshape(2904, 160, 26)

In [27]:
X[np.isnan(X)] = 0

In [28]:
print(np.max(X), np.min(X))

2.0 -1.4914572549019534


In [29]:
# save data into file, a numpy specific format
np.save("X_5.npy",X)
np.save("y_5.npy",y)

In [ ]:
X = np.load("./X_3.npy")
y = np.load("./y_3.npy")

In [30]:
# This is NN LSTM Model creation
def model_lstm(input_shape):
    # The shape was explained above, must have this order
    inp = Input(shape=(input_shape[1], input_shape[2],))
    
    
    # This is the LSTM layer
    # Bidirecional implies that the 160 chunks are calculated in both ways, 0 to 159 and 159 to zero
    # although it appear that just 0 to 159 way matter, I have tested with and without, and tha later worked best
    # 128 and 64 are the number of cells used, too many can overfit and too few can underfit
    x = Bidirectional(LSTM(128, return_sequences=True))(inp)
    print(x.shape)
    # The second LSTM can give more fire power to the model, but can overfit it too
    x = Bidirectional(LSTM(64, return_sequences=True))(x)
    print(x.shape)
    # Attention is a new tecnology that can be applyed to a Recurrent NN to give more meanings to a signal found in the middle
    # of the data, it helps more in longs chains of data. A normal RNN give all the responsibility of detect the signal
    # to the last cell. Google RNN Attention for more information :)
    x = Attention(input_shape[1])(x)
    print(x.shape)
    # A intermediate full connected (Dense) can help to deal with nonlinears outputs
    x = Dense(64)(x)
    print(x.shape)
    
    x = LeakyReLU(alpha=0.1)(x)
    # A binnary classification as this must finish with shape (1,)
    x = Dense(1, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    # Pay attention in the addition of matthews_correlation metric in the compilation, it is a success factor key
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[matthews_correlation])
    
    return model

In [31]:
# Here is where the training happens

# First, create a set of indexes of the 5 folds
splits = list(StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=2019).split(X, y))
preds_val = []
y_val = []
# Then, iteract with each fold
# If you dont know, enumerate(['a', 'b', 'c']) returns [(0, 'a'), (1, 'b'), (2, 'c')]
for idx, (train_idx, val_idx) in enumerate(splits):
    K.clear_session() # I dont know what it do, but I imagine that it "clear session" :)
    print("Beginning fold {}".format(idx+1))
    # use the indexes to extract the folds in the train and validation data
    train_X, train_y, val_X, val_y = X[train_idx], y[train_idx], X[val_idx], y[val_idx]
    # instantiate the model for this fold
    model = model_lstm(train_X.shape)
    # This checkpoint helps to avoid overfitting. It just save the weights of the model if it delivered an
    # validation matthews_correlation greater than the last one.
    ckpt = ModelCheckpoint('weights_{}.h5'.format(idx), save_best_only=True, save_weights_only=True, verbose=1, monitor='val_matthews_correlation', mode='max')
    # Train, train, train
    model.fit(train_X, train_y, batch_size=128, epochs=50, validation_data=[val_X, val_y], callbacks=[ckpt])
    # loads the best weights saved by the checkpoint
    model.load_weights('weights_{}.h5'.format(idx))
    # Add the predictions of the validation to the list preds_val
    preds_val.append(model.predict(val_X, batch_size=512))
    # and the val true y
    y_val.append(val_y)

# concatenates all and prints the shape    
preds_val = np.concatenate(preds_val)[...,0]
y_val = np.concatenate(y_val)
preds_val.shape, y_val.shape

Beginning fold 1
(?, ?, 256)
(?, ?, 128)
(?, 128)
(?, 64)
Train on 2322 samples, validate on 582 samples
Epoch 1/50
2322/2322 [==============================] - 27s 11ms/step - loss: 0.2965 - matthews_correlation: 0.0049 - val_loss: 0.2256 - val_matthews_correlation: 0.0000e+00

Epoch 00001: val_matthews_correlation improved from -inf to 0.00000, saving model to weights_0.h5
Epoch 2/50
2322/2322 [==============================] - 20s 9ms/step - loss: 0.2092 - matthews_correlation: 0.0701 - val_loss: 0.2061 - val_matthews_correlation: 0.1923

Epoch 00002: val_matthews_correlation improved from 0.00000 to 0.19231, saving model to weights_0.h5
Epoch 3/50
2322/2322 [==============================] - 20s 9ms/step - loss: 0.1930 - matthews_correlation: 0.2857 - val_loss: 0.2262 - val_matthews_correlation: 0.2192

Epoch 00003: val_matthews_correlation improved from 0.19231 to 0.21918, saving model to weights_0.h5
Epoch 4/50
2322/2322 [==============================] - 20s 9ms/step - loss: 0.1

2322/2322 [==============================] - 20s 9ms/step - loss: 0.1034 - matthews_correlation: 0.6180 - val_loss: 0.0949 - val_matthews_correlation: 0.7299

Epoch 00033: val_matthews_correlation improved from 0.70696 to 0.72992, saving model to weights_0.h5
Epoch 34/50
2322/2322 [==============================] - 20s 9ms/step - loss: 0.1051 - matthews_correlation: 0.6411 - val_loss: 0.0933 - val_matthews_correlation: 0.7072

Epoch 00034: val_matthews_correlation did not improve from 0.72992
Epoch 35/50
2322/2322 [==============================] - 20s 9ms/step - loss: 0.1027 - matthews_correlation: 0.6400 - val_loss: 0.1027 - val_matthews_correlation: 0.6694

Epoch 00035: val_matthews_correlation did not improve from 0.72992
Epoch 36/50
2322/2322 [==============================] - 20s 9ms/step - loss: 0.1009 - matthews_correlation: 0.6445 - val_loss: 0.0949 - val_matthews_correlation: 0.6730

Epoch 00036: val_matthews_correlation did not improve from 0.72992
Epoch 37/50
2322/2322 [===

2323/2323 [==============================] - 20s 9ms/step - loss: 0.1220 - matthews_correlation: 0.4214 - val_loss: 0.1428 - val_matthews_correlation: 0.5831

Epoch 00016: val_matthews_correlation did not improve from 0.62775
Epoch 17/50
2323/2323 [==============================] - 20s 9ms/step - loss: 0.1238 - matthews_correlation: 0.5479 - val_loss: 0.1231 - val_matthews_correlation: 0.5572

Epoch 00017: val_matthews_correlation did not improve from 0.62775
Epoch 18/50
2323/2323 [==============================] - 20s 9ms/step - loss: 0.1176 - matthews_correlation: 0.5427 - val_loss: 0.1267 - val_matthews_correlation: 0.6364

Epoch 00018: val_matthews_correlation improved from 0.62775 to 0.63640, saving model to weights_1.h5
Epoch 19/50
2323/2323 [==============================] - 21s 9ms/step - loss: 0.1149 - matthews_correlation: 0.5542 - val_loss: 0.1086 - val_matthews_correlation: 0.6415

Epoch 00019: val_matthews_correlation improved from 0.63640 to 0.64151, saving model to weigh

2323/2323 [==============================] - 21s 9ms/step - loss: 0.0808 - matthews_correlation: 0.7065 - val_loss: 0.1951 - val_matthews_correlation: 0.4065

Epoch 00050: val_matthews_correlation did not improve from 0.67809
Beginning fold 3
(?, ?, 256)
(?, ?, 128)
(?, 128)
(?, 64)
Train on 2323 samples, validate on 581 samples
Epoch 1/50
2323/2323 [==============================] - 24s 11ms/step - loss: 0.3110 - matthews_correlation: 0.0000e+00 - val_loss: 0.2342 - val_matthews_correlation: 0.0000e+00

Epoch 00001: val_matthews_correlation improved from -inf to 0.00000, saving model to weights_2.h5
Epoch 2/50
2323/2323 [==============================] - 20s 9ms/step - loss: 0.2289 - matthews_correlation: 0.0000e+00 - val_loss: 0.2218 - val_matthews_correlation: 0.0000e+00

Epoch 00002: val_matthews_correlation did not improve from 0.00000
Epoch 3/50
2323/2323 [==============================] - 20s 9ms/step - loss: 0.2156 - matthews_correlation: 0.0442 - val_loss: 0.2054 - val_matthew


Epoch 00032: val_matthews_correlation did not improve from 0.70102
Epoch 33/50
2323/2323 [==============================] - 20s 9ms/step - loss: 0.1045 - matthews_correlation: 0.6115 - val_loss: 0.1107 - val_matthews_correlation: 0.4854

Epoch 00033: val_matthews_correlation did not improve from 0.70102
Epoch 34/50
2323/2323 [==============================] - 20s 9ms/step - loss: 0.1012 - matthews_correlation: 0.6025 - val_loss: 0.1099 - val_matthews_correlation: 0.5478

Epoch 00034: val_matthews_correlation did not improve from 0.70102
Epoch 35/50
2323/2323 [==============================] - 20s 9ms/step - loss: 0.1001 - matthews_correlation: 0.6433 - val_loss: 0.1095 - val_matthews_correlation: 0.5149

Epoch 00035: val_matthews_correlation did not improve from 0.70102
Epoch 36/50
2323/2323 [==============================] - 20s 9ms/step - loss: 0.1065 - matthews_correlation: 0.6198 - val_loss: 0.1156 - val_matthews_correlation: 0.4734

Epoch 00036: val_matthews_correlation did not i

2324/2324 [==============================] - 20s 9ms/step - loss: 0.1241 - matthews_correlation: 0.4996 - val_loss: 0.1246 - val_matthews_correlation: 0.5142

Epoch 00016: val_matthews_correlation improved from 0.46038 to 0.51421, saving model to weights_3.h5
Epoch 17/50
2324/2324 [==============================] - 20s 9ms/step - loss: 0.1175 - matthews_correlation: 0.5657 - val_loss: 0.1328 - val_matthews_correlation: 0.5375

Epoch 00017: val_matthews_correlation improved from 0.51421 to 0.53754, saving model to weights_3.h5
Epoch 18/50
2324/2324 [==============================] - 20s 9ms/step - loss: 0.1207 - matthews_correlation: 0.5333 - val_loss: 0.1222 - val_matthews_correlation: 0.5347

Epoch 00018: val_matthews_correlation did not improve from 0.53754
Epoch 19/50
2324/2324 [==============================] - 20s 9ms/step - loss: 0.1201 - matthews_correlation: 0.5471 - val_loss: 0.1263 - val_matthews_correlation: 0.5494

Epoch 00019: val_matthews_correlation improved from 0.53754

2324/2324 [==============================] - 21s 9ms/step - loss: 0.0976 - matthews_correlation: 0.6206 - val_loss: 0.1213 - val_matthews_correlation: 0.5616

Epoch 00049: val_matthews_correlation did not improve from 0.69522
Epoch 50/50
2324/2324 [==============================] - 21s 9ms/step - loss: 0.0907 - matthews_correlation: 0.6664 - val_loss: 0.1221 - val_matthews_correlation: 0.5649

Epoch 00050: val_matthews_correlation did not improve from 0.69522
Beginning fold 5
(?, ?, 256)
(?, ?, 128)
(?, 128)
(?, 64)
Train on 2324 samples, validate on 580 samples
Epoch 1/50
2324/2324 [==============================] - 25s 11ms/step - loss: 0.3003 - matthews_correlation: 0.0015 - val_loss: 0.2341 - val_matthews_correlation: 0.0000e+00

Epoch 00001: val_matthews_correlation improved from -inf to 0.00000, saving model to weights_4.h5
Epoch 2/50
2324/2324 [==============================] - 21s 9ms/step - loss: 0.2344 - matthews_correlation: 0.0000e+00 - val_loss: 0.1987 - val_matthews_corre


Epoch 00031: val_matthews_correlation improved from 0.56665 to 0.57956, saving model to weights_4.h5
Epoch 32/50
2324/2324 [==============================] - 20s 9ms/step - loss: 0.1041 - matthews_correlation: 0.6294 - val_loss: 0.1217 - val_matthews_correlation: 0.4030

Epoch 00032: val_matthews_correlation did not improve from 0.57956
Epoch 33/50
2324/2324 [==============================] - 20s 9ms/step - loss: 0.1037 - matthews_correlation: 0.6176 - val_loss: 0.1281 - val_matthews_correlation: 0.5285

Epoch 00033: val_matthews_correlation did not improve from 0.57956
Epoch 34/50
2324/2324 [==============================] - 20s 9ms/step - loss: 0.1058 - matthews_correlation: 0.6248 - val_loss: 0.1308 - val_matthews_correlation: 0.3960

Epoch 00034: val_matthews_correlation did not improve from 0.57956
Epoch 35/50
2324/2324 [==============================] - 20s 9ms/step - loss: 0.1170 - matthews_correlation: 0.5646 - val_loss: 0.1322 - val_matthews_correlation: 0.4469

Epoch 00035: 

((2904,), (2904,))

In [32]:
np.save('./tmp/y_val_tmp_7.npy', y_val)
np.save('./tmp/preds_val_tmp_7.npy', preds_val)

In [33]:
# It is the official metric used in this competition
# below is the declaration of a function used inside the keras model, calculation with K (keras backend / thensorflow)
def matthews_correlation(y_true, y_pred):
    '''Calculates the Matthews correlation coefficient measure for quality
    of binary classification problems.
    '''
    
    #y_pred = K.cast(y_pred, np.float)
    y_pred_pos = np.round(np.clip(y_pred, 0, 1))
    y_pred_neg = 1 - y_pred_pos

    y_pos = np.round(np.clip(y_true, 0, 1))
    y_neg = 1 - y_pos

    tp = np.sum(y_pos * y_pred_pos)
    tn = np.sum(y_neg * y_pred_neg)

    fp = np.sum(y_neg * y_pred_pos)
    fn = np.sum(y_pos * y_pred_neg)

    numerator = (tp * tn - fp * fn)
    denominator = np.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))

    return numerator / (denominator + K.epsilon())

In [34]:
# The output of this kernel must be binary (0 or 1), but the output of the NN Model is float (0 to 1).
# So, find the best threshold to convert float to binary is crucial to the result
# this piece of code is a function that evaluates all the possible thresholds from 0 to 1 by 0.01
def threshold_search(y_true, y_proba):
    best_threshold = 0
    best_score = 0
    for threshold in tqdm([i * 0.01 for i in range(100)]):
        #score = matthews_correlation(y_true, (y_proba > threshold).astype(int))
        #score = K.eval(matthews_correlation(y_true, (y_proba > threshold).astype(int)))
        #score = K.eval(matthews_correlation(y_true.astype(np.float64), (y_proba > threshold).astype(np.float64)))
        score = matthews_correlation(y_true.astype(np.float64), (y_proba > threshold).astype(np.float64))
        if score > best_score:
            best_threshold = threshold
            best_score = score
    search_result = {'threshold': best_threshold, 'matthews_correlation': best_score}
    return search_result

In [35]:
best_threshold = threshold_search(y_val, preds_val)['threshold']

100%|██████████| 100/100 [00:00<00:00, 10804.21it/s]


In [36]:
best_threshold

0.5

In [37]:
matthews_correlation(y_val[:582], preds_val[:582])

0.7014011906884012

In [38]:
matthews_correlation(y_val, preds_val)

0.6704652019341645

In [49]:
threshold_search(y_val[:582], preds_val[:582])['threshold']

100%|██████████| 100/100 [00:00<00:00, 14573.68it/s]


0.5

In [39]:
%%time
# Now load the test data
# This first part is the meta data, not the main data, the measurements
meta_test = pd.read_csv('../input/metadata_test.csv')
df_test = meta_test.set_index(['id_measurement', 'phase'])

CPU times: user 3.5 ms, sys: 11.2 ms, total: 14.7 ms
Wall time: 36.6 ms


In [40]:
meta_test = meta_test.set_index(['signal_id'])
meta_test.head()

,id_measurement,phase
signal_id,,
8712,2904,0
8713,2904,1
8714,2904,2
8715,2905,0
8716,2905,1


In [41]:
# this function take a piece of data and convert using transform_ts(), but it does to each of the 3 phases
# if we would try to do in one time, could exceed the RAM Memmory
def prep_data_test(start, end):
    # load a piece of data from file
    praq_test = pq.read_pandas('../input/test.parquet', columns=[str(i) for i in range(start+8712, end+8712)]).to_pandas()
    X = []

    # using tdqm to evaluate processing time
    # takes each index from df_train and iteract it from start to end
    # it is divided by 3 because for each id_measurement there are 3 id_signal, and the start/end parameters are id_signal
    for id_measurement in tqdm(df_test.index.levels[0].unique()[int(start/3):int(end/3)]):
        X_signal = []
        # for each phase of the signal
        for phase in [0,1,2]:
            # extract from df_train both signal_id and target to compose the new data sets
            signal_id = df_test.loc[id_measurement].loc[phase][0]
            # but just append the target one time, to not triplicate it
            if phase == 0:
                ts_sum = praq_test[str(signal_id)] + praq_test[str(signal_id+1)] + praq_test[str(signal_id+2)]
                X_signal.append(transform_ts_sum(ts_sum))
            # extract and transform data into sets of features
            #X_signal.append(transform_ts(praq_test[str(signal_id)]))
        # concatenate all the 3 phases in one matrix
        X_signal = np.concatenate(X_signal, axis=1)
        # add the data to X
        X.append(X_signal)
    X = np.asarray(X)
    return X

In [42]:
def process_subtest(arg_tuple):
    start, end, idx = arg_tuple
    X = prep_data_test(start, end)
    return idx, X

In [43]:
all_chunks = []

num_cores = 16 
#def load_all():
total_size = len(meta_test)
chunk_size = np.ceil(total_size/num_cores)
#train_size = len(df_train)

for i in range(16):
    if i != 15:
        start_idx = int(i * chunk_size)
        end_idx = int(start_idx + chunk_size)
        #chunk = (start_idx+train_size, end_idx+train_size, i)
        chunk = (start_idx, end_idx, i)
        all_chunks.append(chunk)
    else:
        start_idx = int(i * chunk_size)
        end_idx = int(total_size)
        #chunk = (start_idx+train_size, end_idx+train_size, i)
        chunk = (start_idx, end_idx, i)
        all_chunks.append(chunk)
        

In [44]:
pool = Pool()
results_1 = pool.map(process_subtest, all_chunks[0:8])    
results_1 = sorted(results_1, key=lambda tup: tup[0])

100%|██████████| 424/424 [05:48<00:00,  1.72it/s]


In [45]:
results_2 = pool.map(process_subtest, all_chunks[8:16])    
results_2 = sorted(results_2, key=lambda tup: tup[0])

In [46]:
results = results_1 + results_2
X_test = np.concatenate([item[1] for item in results], axis=0)

In [47]:
np.save("X_test_5.npy",X_test)

In [48]:
submission = pd.read_csv('../input/sample_submission.csv')
print(len(submission))
submission.head()

20337


,signal_id,target
0,8712,0
1,8713,0
2,8714,0
3,8715,0
4,8716,0


splits = list(StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=2019).split(X, y))

for idx, (train_idx, val_idx) in enumerate(splits):
    K.clear_session() # I dont know what it do, but I imagine that it "clear session" :)
    print("Beginning fold {}".format(idx+1))
    # use the indexes to extract the folds in the train and validation data
    train_X, train_y, val_X, val_y = X[train_idx], y[train_idx], X[val_idx], y[val_idx]
    # instantiate the model for this fold
    model = model_lstm(train_X.shape)

In [49]:
X_test[np.isnan(X_test)] = 0

In [50]:
preds_test = []
for i in range(N_SPLITS):
    model.load_weights('weights_{}.h5'.format(i))
    pred = model.predict(X_test, batch_size=300, verbose=1)
    pred_3 = []
    for pred_scalar in pred:
        for i in range(3):
            pred_3.append(pred_scalar)
    preds_test.append(pred_3)

6779/6779 [==============================] - 16s 2ms/step


In [51]:
preds_test = (np.squeeze(np.mean(preds_test, axis=0)) > best_threshold).astype(np.int)
preds_test.shape

(20337,)

In [53]:
submission['target'] = preds_test
submission.to_csv('../output/submission_23.csv', index=False)
submission.head()

,signal_id,target
0,8712,0
1,8713,0
2,8714,0
3,8715,0
4,8716,0


In [ ]:
lstm_preds = np.squeeze(np.mean(preds_test, axis=0))
np.save("./lstm_preds.npy", lstm_preds)

In [52]:
np.sum(preds_test[preds_test==1])

663

In [59]:
submission['target'].value_counts()

0    19404
1      933
Name: target, dtype: int64

In [60]:
model.load_weights('weights_0.h5')
pred = model.predict(X_test, batch_size=300, verbose=1)
pred_3 = []
for pred_scalar in pred:
    for i in range(3):
        pred_3.append(pred_scalar)

6779/6779 [==============================] - 17s 2ms/step


In [62]:
pred_3>0.5

TypeError: '>' not supported between instances of 'list' and 'float'

In [65]:
preds_3 = (np.squeeze(pred_3) > 0.5).astype(np.int)

In [66]:
np.sum(preds_3[preds_3==1])

1209

In [69]:
submission['target'] = preds_3
submission.to_csv('../output/submission_22.csv', index=False)

In [68]:
submission['target'].value_counts()

0    19128
1     1209
Name: target, dtype: int64